In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [2]:
DATA_PATH = Path("./data")
TRAIN_PATH = DATA_PATH / "train.csv"
TEST_PATH = DATA_PATH / "test.csv"
SAMPLE_SUBMISSION_PATH = DATA_PATH / "gender_submission.csv"

In [3]:
train_df = pd.read_csv(TRAIN_PATH)

print(len(train_df))
train_df.head()

891


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
test_df = pd.read_csv(TEST_PATH)
test_ids = test_df["PassengerId"]


print(len(test_df))
test_df.head()

418


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


## EDA

In [5]:
train_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
train_df.describe()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
count,891.000000,891.000000,891.000000,714.000000,891.000000,891.000000,891.000000
mean,446.000000,0.383838,2.308642,29.699118,0.523008,0.381594,32.204208
std,257.353842,0.486592,0.836071,14.526497,1.102743,0.806057,49.693429
min,1.000000,0.000000,1.000000,0.420000,0.000000,0.000000,0.000000
25%,223.500000,0.000000,2.000000,20.125000,0.000000,0.000000,7.910400
50%,446.000000,0.000000,3.000000,28.000000,0.000000,0.000000,14.454200
75%,668.500000,1.000000,3.000000,38.000000,1.000000,0.000000,31.000000
max,891.000000,1.000000,3.000000,80.000000,8.000000,6.000000,512.329200


In [7]:
train_df.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [8]:
men = train_df.loc[train_df.Sex == "male"]["Survived"]
rate_men = sum(men)/len(men)

women = train_df.loc[train_df.Sex == "female"]["Survived"]
rate_women = sum(women)/len(women)

print("% of men who survived:", rate_men)
print("% of women who survived:", rate_women)

% of men who survived: 0.18890814558058924
% of women who survived: 0.7420382165605095


In [9]:
# Name
pre_names = train_df["Name"].str.split(", ", expand=True)[1].str.split(".", expand=True)[0].unique()

pre_names

array(['Mr', 'Mrs', 'Miss', 'Master', 'Don', 'Rev', 'Dr', 'Mme', 'Ms',
       'Major', 'Lady', 'Sir', 'Mlle', 'Col', 'Capt', 'the Countess',
       'Jonkheer'], dtype=object)

In [10]:
# Cabin
print(train_df.loc[train_df["Cabin"].notna()]["Cabin"][:10])
print("\n", train_df["Cabin"].str.slice(0,1).unique())
print("\n", train_df["Cabin"].str.slice(0,1).value_counts())
print("\n", "Nan values:", train_df["Cabin"].isna().sum(), "out of", len(train_df), "total rows")

1             C85
3            C123
6             E46
10             G6
11           C103
21            D56
23             A6
27    C23 C25 C27
31            B78
52            D33
Name: Cabin, dtype: object

 [nan 'C' 'E' 'G' 'D' 'A' 'B' 'F' 'T']

 C    59
B    47
D    33
E    32
A    15
F    13
G     4
T     1
Name: Cabin, dtype: int64

 Nan values: 687 out of 891 total rows


## Data Preprocessing

In [11]:
def clean1(data):
    data = data.drop(["Ticket", "Cabin", "Name", "PassengerId"], axis=1)

    cols = ["SibSp", "Parch", "Fare", "Age"]
    for col in cols:
        data[col].fillna(data[col].median(), inplace=True)

    data.Embarked.fillna("U", inplace=True)

    return data


post1_train_df = clean1(train_df)
post1_test_df = clean1(test_df)

print(len(post1_train_df), len(post1_test_df))

891 418


In [12]:
le = preprocessing.LabelEncoder()

cols = ["Sex", "Embarked"]

for col in cols:
    post1_train_df[col] = le.fit_transform (post1_train_df[col])
    post1_test_df[col] = le.transform(post1_test_df[col])
    print(le.classes_)


['female' 'male']
['C' 'Q' 'S' 'U']


In [13]:
post1_train_df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,1,22.0,1,0,7.2500,2
1,1,1,0,38.0,1,0,71.2833,0
2,1,3,0,26.0,0,0,7.9250,2
3,1,1,0,35.0,1,0,53.1000,2
4,0,3,1,35.0,0,0,8.0500,2


In [14]:
y = post1_train_df["Survived"]
X = post1_train_df.drop("Survived", axis=1)

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Model Training

#### LogisticRegression (sklearn)

In [15]:
model1 = LogisticRegression(random_state=0, max_iter=1000).fit(X_train, y_train)

In [16]:
predictions = model1.predict(X_val)

accuracy_score(y_val, predictions)

0.8100558659217877

In [17]:
submission_preds = model1.predict(post1_test_df)

In [37]:
submission_df = pd.DataFrame({"PassengerId": test_ids.values,
                            "Survived": submission_preds
                            })

submission_df.to_csv("submission.csv", index=False)

#### XGBoost Classifier

In [21]:
model2 = xgb.XGBClassifier().fit(X_train, y_train)

In [22]:
# Make predictions on validation set
y_pred = model2.predict(X_val)

# Evaluate accuracy
accuracy = accuracy_score(y_val, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.79


In [23]:
submission_preds = model2.predict(post1_test_df)

In [24]:
submission_df = pd.DataFrame({"PassengerId": test_ids.values,
                            "Survived": submission_preds
                            })

submission_df.to_csv("submission.csv", index=False)